In [24]:
import pyodbc
import pandas as pd
from sqlalchemy import create_engine

# Define the connection string
conn_str = (
    'Driver={ODBC Driver 17 for SQL Server};'
    'Server=azwsynt00.sql.azuresynapse.net;'
    'Database=AZWSYNT00;'
    'Authentication=ActiveDirectoryIntegrated;'
)

# Use SQLAlchemy to create the engine
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={conn_str}')

# Define the SQL query
query = '''
WITH latest_PPA AS (
    SELECT
        dad.ACCOUNTNUMBER,
        DPP.writtendate,
        paymentplankey,
        ROW_NUMBER() OVER (PARTITION BY dad.accountnumber ORDER BY dpp.writtendate DESC) AS rows
    FROM dw.ILMS_DIM_ACCOUNT_DETAIL dad
    LEFT JOIN dw.ILMS_DIM_PAYMENT_PLAN dpp
        ON dpp.ACCOUNTDETAILKEY = dad.accountdetailkey
),
latest_PPA1 AS (
    SELECT *
    FROM latest_PPA
    WHERE rows = 1
)
SELECT
    CONVERT(VARCHAR, dppd.duedate, 23) AS duedate,  -- Converting to YYYY-MM-DD format
    CASE 
        WHEN dppd.paymentmethod IN ('Debit Card', 'Credit Card') THEN 'Card'
        ELSE dppd.paymentmethod
    END AS paymentmethod,
    SUM(dppd.paymentamount) AS total_payment_amount,
    COUNT(dppd.paymentamount) AS item_count  -- adding count of items per day
FROM latest_PPA1 dpp
INNER JOIN dw.ILMS_DIM_PAYMENT_PLAN_DETAIL dppd
    ON dppd.PAYMENTPLANKEY = dpp.PAYMENTPLANKEY
WHERE dppd.duedate BETWEEN '2023-09-01' AND CAST(GETDATE() AS DATE)
  AND (CASE 
        WHEN dppd.paymentmethod IN ('Debit Card', 'Credit Card') THEN 'Card'
        ELSE dppd.paymentmethod
     END) IN ('Card', 'ACH')
GROUP BY CONVERT(VARCHAR, dppd.duedate, 23), CASE 
        WHEN dppd.paymentmethod IN ('Debit Card', 'Credit Card') THEN 'Card'
        ELSE dppd.paymentmethod
     END
ORDER BY duedate;
'''

# Execute the query and fetch the results into a DataFrame
results_df = pd.read_sql(query, engine)

# Rename the columns
results_df.columns = ['Date', 'Payment Method', 'Total Amount', 'Total Count']

# Split the result into two DataFrames based on 'paymentmethod'
ilms_card = results_df[results_df['Payment Method'] == 'Card']
ilms_ach = results_df[results_df['Payment Method'] == 'ACH']

# Display the DataFrames for verification
print("ILMS Card DataFrame:")
print(ilms_card.head())

print("ILMS ACH DataFrame:")
print(ilms_ach.tail())

ILMS Card DataFrame:
         Date Payment Method  Total Amount  Total Count
0  2023-09-01           Card     226028.26         1994
3  2023-09-02           Card      86299.20          643
4  2023-09-03           Card      66199.06          873
6  2023-09-04           Card      73891.70          789
8  2023-09-05           Card     189253.90         1090
ILMS ACH DataFrame:
           Date Payment Method  Total Amount  Total Count
546  2024-05-31            ACH    1434128.18         9354
548  2024-06-01            ACH     273265.50         2574
550  2024-06-02            ACH      93146.36          921
553  2024-06-03            ACH     274006.18         2053
555  2024-06-04            ACH     111077.70         1236


In [30]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet

# Split the result into two DataFrames based on 'paymentmethod'
ilms_card = results_df[results_df['Payment Method'] == 'Card'].copy()
ilms_ach = results_df[results_df['Payment Method'] == 'ACH'].copy()

# Function for predictive modeling

def predictive_modeling(df):
    # Convert the 'Date' column to datetime in the correct format
    df.loc[:, 'Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')  # Fix for SettingWithCopyWarning
    
    # Ensure the index is set correctly after date conversion
    df.set_index('Date', inplace=True)
    
    data = df['Total Count']

    # Prophet model
    df_prophet = df.reset_index().rename(columns={'Date': 'ds', 'Total Count': 'y'})
    model = Prophet(seasonality_mode='multiplicative', seasonality_prior_scale=1.5)
    model.add_seasonality(name='weekly', period=7, fourier_order=5)
    model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    model.add_seasonality(name='yearly', period=365.25, fourier_order=10)
    model.fit(df_prophet)
    future = model.make_future_dataframe(periods=1, freq='D')
    forecast = model.predict(future)
    prophet_prediction = forecast.iloc[-2]['yhat']  # Changed from -1 to -2 to make the prediction on yesterday

    prediction_date = df.index.max()  # Changed from df.index.max() + pd.Timedelta(days=1)

    # Calculate the average of the last 4 similar weekdays
    weekdays_data = df[df.index.weekday == prediction_date.weekday()]['Total Count']
    last_4_similar_weekdays = weekdays_data.tail(4).mean() if len(weekdays_data) > 0 else None

    return prediction_date, prophet_prediction, last_4_similar_weekdays

# Run predictive modeling for both dataframes
pred_date_card, prophet_card, avg_weekdays_card = predictive_modeling(ilms_card)
pred_date_ach, prophet_ach, avg_weekdays_ach = predictive_modeling(ilms_ach)

# Reset the index to reintroduce 'Date' column for comparison
ilms_card.reset_index(inplace=True)
ilms_ach.reset_index(inplace=True)

# Fetch actual values for the prediction dates
actual_card = ilms_card[ilms_card['Date'] == pred_date_card]['Total Count']  # Changed from pred_date_card - pd.Timedelta(days=1)
actual_ach = ilms_ach[ilms_ach['Date'] == pred_date_ach]['Total Count']  # Changed from pred_date_ach - pd.Timedelta(days=1)

actual_card = actual_card.values[0] if not actual_card.empty else None
actual_ach = actual_ach.values[0] if not actual_ach.empty else None

# Create DataFrames for predictions and actual values
card_predictions = pd.DataFrame({
    'Date': [pred_date_card],
    'Prophet Prediction': [prophet_card],
    'Average of the Last 4 Similar Weekdays': [avg_weekdays_card],
    'Actual': [actual_card],
    'Closer Prediction': [prophet_card if actual_card is not None and abs(prophet_card - actual_card) < abs(avg_weekdays_card - actual_card) else avg_weekdays_card]
})
card_predictions['Percentage Difference'] = (abs(card_predictions['Actual'] - card_predictions['Closer Prediction']) / card_predictions['Actual'])

ach_predictions = pd.DataFrame({
    'Date': [pred_date_ach],
    'Prophet Prediction': [prophet_ach],
    'Average of the Last 4 Similar Weekdays': [avg_weekdays_ach],
    'Actual': [actual_ach],
    'Closer Prediction': [prophet_ach if actual_ach is not None and abs(prophet_ach - actual_ach) < abs(avg_weekdays_ach - actual_ach) else avg_weekdays_ach]
})
ach_predictions['Percentage Difference'] = (abs(ach_predictions['Actual'] - ach_predictions['Closer Prediction']) / ach_predictions['Actual'])

# Display the DataFrames for verification
print("ILMS Card Predictions DataFrame:")
print(card_predictions.head())

print("ILMS ACH Predictions DataFrame:")
print(ach_predictions.head())

# Export DataFrames to CSV
card_predictions.to_csv(r'C:\Users\pskotte\Desktop\Power Bi Data\ILMSCardPredictions.csv', index=False)
ach_predictions.to_csv(r'C:\Users\pskotte\Desktop\Power Bi Data\ILMSACHPredictions.csv', index=False)

13:55:28 - cmdstanpy - INFO - Chain [1] start processing
13:55:28 - cmdstanpy - INFO - Chain [1] done processing
13:55:29 - cmdstanpy - INFO - Chain [1] start processing
13:55:29 - cmdstanpy - INFO - Chain [1] done processing


ILMS Card Predictions DataFrame:
        Date  Prophet Prediction  Average of the Last 4 Similar Weekdays  \
0 2024-06-04         1068.552635                                 1096.75   

   Actual  Closer Prediction  Percentage Difference  
0    1112            1096.75               0.013714  
ILMS ACH Predictions DataFrame:
        Date  Prophet Prediction  Average of the Last 4 Similar Weekdays  \
0 2024-06-04         1067.252095                                  1410.0   

   Actual  Closer Prediction  Percentage Difference  
0    1236        1067.252095               0.136527  
